In [1]:
import pandas as pd

In [2]:
import glob

csv_files = glob.glob("DATA/*.csv")

In [5]:
data_list = []
for csv in csv_files:
    csv = csv.replace('.csv', '')
    tname = csv.split('\\')[-1]
    dbname = tname.split('_')[0]
    itemname = tname.split('_')[1]
    data_list.append({
        'dbname': dbname,
        'itemname': itemname,
    })


In [6]:
data_list

[{'dbname': 'ECOS', 'itemname': 'KRW'},
 {'dbname': 'FRED', 'itemname': 'DCOILWTICO'},
 {'dbname': 'FRED', 'itemname': 'DEXCHUS'},
 {'dbname': 'FRED', 'itemname': 'DEXJPUS'},
 {'dbname': 'FRED', 'itemname': 'DEXUSEU'},
 {'dbname': 'FRED', 'itemname': 'DGS10'},
 {'dbname': 'FRED', 'itemname': 'DTWEXBGS'},
 {'dbname': 'FRED', 'itemname': 'SP500'},
 {'dbname': 'XBBG', 'itemname': 'CNY'},
 {'dbname': 'XBBG', 'itemname': 'DXY'},
 {'dbname': 'XBBG', 'itemname': 'GOLD'},
 {'dbname': 'XBBG', 'itemname': 'NICKEL'},
 {'dbname': 'XBBG', 'itemname': 'SP500'},
 {'dbname': 'XBBG', 'itemname': 'USGG10YR'},
 {'dbname': 'XBBG', 'itemname': 'WTI'}]

In [11]:
df = pd.read_csv(csv_files[0], parse_dates=["date"])
df.dtypes

date     datetime64[ns]
value           float64
name             object
dtype: object

In [12]:
df

,date,value,name
0,1964-05-04,255.77,KRW
1,1964-05-05,255.77,KRW
2,1964-05-06,255.77,KRW
3,1964-05-07,255.77,KRW
4,1964-05-08,255.77,KRW
...,...,...,...
16377,2025-06-02,1373.10,KRW
16378,2025-06-04,1369.50,KRW
16379,2025-06-05,1358.40,KRW
16380,2025-06-09,1356.40,KRW


In [13]:
import pymysql

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
host = os.getenv("db_host")
user = os.getenv("db_user")
password = os.getenv("db_password")
database = os.getenv("db_database")

In [ ]:
# 2. DB 접속 정보
conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database,
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()


In [20]:
def insert_dataframe_to_mysql(table_name: str, df: pd.DataFrame, conn):
    """
    DataFrame을 MySQL에 삽입하는 함수.
    테이블이 없으면 생성하고, 있으면 그대로 INSERT 수행.
    
    :param table_name: 생성 또는 삽입할 테이블 이름
    :param df: pandas DataFrame (columns: date, value, item)
    :param conn: pymysql 커넥션 객체
    """
    cursor = conn.cursor()

    # 1. 테이블 존재 여부 확인
    check_sql = """
        SELECT COUNT(*) AS count 
        FROM information_schema.tables 
        WHERE table_schema = DATABASE() AND table_name = %s
    """
    cursor.execute(check_sql, (table_name,))
    result = cursor.fetchone()

    # 2. 테이블이 없다면 생성
    if result["count"] == 0:
        create_sql = f"""
        CREATE TABLE {table_name} (
            date DATE,
            value DOUBLE,
            item VARCHAR(255)
        )
        """
        cursor.execute(create_sql)

    # 3. 데이터 INSERT
    insert_sql = f"INSERT INTO {table_name} (date, value, item) VALUES (%s, %s, %s)"
    records = list(df.itertuples(index=False, name=None))  # 튜플 리스트
    cursor.executemany(insert_sql, records)

    conn.commit()
    cursor.close()

In [21]:
csv_files, data_list

(['DATA\\ECOS_KRW.csv',
  'DATA\\FRED_DCOILWTICO.csv',
  'DATA\\FRED_DEXCHUS.csv',
  'DATA\\FRED_DEXJPUS.csv',
  'DATA\\FRED_DEXUSEU.csv',
  'DATA\\FRED_DGS10.csv',
  'DATA\\FRED_DTWEXBGS.csv',
  'DATA\\FRED_SP500.csv',
  'DATA\\XBBG_CNY.csv',
  'DATA\\XBBG_DXY.csv',
  'DATA\\XBBG_GOLD.csv',
  'DATA\\XBBG_NICKEL.csv',
  'DATA\\XBBG_SP500.csv',
  'DATA\\XBBG_USGG10YR.csv',
  'DATA\\XBBG_WTI.csv'],
 [{'dbname': 'ECOS', 'itemname': 'KRW'},
  {'dbname': 'FRED', 'itemname': 'DCOILWTICO'},
  {'dbname': 'FRED', 'itemname': 'DEXCHUS'},
  {'dbname': 'FRED', 'itemname': 'DEXJPUS'},
  {'dbname': 'FRED', 'itemname': 'DEXUSEU'},
  {'dbname': 'FRED', 'itemname': 'DGS10'},
  {'dbname': 'FRED', 'itemname': 'DTWEXBGS'},
  {'dbname': 'FRED', 'itemname': 'SP500'},
  {'dbname': 'XBBG', 'itemname': 'CNY'},
  {'dbname': 'XBBG', 'itemname': 'DXY'},
  {'dbname': 'XBBG', 'itemname': 'GOLD'},
  {'dbname': 'XBBG', 'itemname': 'NICKEL'},
  {'dbname': 'XBBG', 'itemname': 'SP500'},
  {'dbname': 'XBBG', 'itemname': 

In [22]:
for csv_path, data_info in zip(csv_files, data_list):
    df = pd.read_csv(csv_path, parse_dates=["date"])
    insert_dataframe_to_mysql(data_info["dbname"], df, conn)


In [23]:

conn.close()

In [27]:
pd.read_csv("http://10.1.13.178:5000/fred/DTWEXBGS/")

,date,value,item
0,2006-01-02,101.4155,DTWEXBGS
1,2006-01-03,100.7558,DTWEXBGS
2,2006-01-04,100.2288,DTWEXBGS
3,2006-01-05,100.2992,DTWEXBGS
4,2006-01-06,100.0241,DTWEXBGS
...,...,...,...
4867,2025-06-02,121.5490,DTWEXBGS
4868,2025-06-03,121.8080,DTWEXBGS
4869,2025-06-04,121.4093,DTWEXBGS
4870,2025-06-05,121.3178,DTWEXBGS
